<a href="https://colab.research.google.com/github/tonykipkemboi/Gap_Inc._Virtual_Learning_Experience_COVID_DS_Project/blob/updates/GAP_Inc_COVID_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LOAD DATA


##Dependencies

In [189]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from datetime import datetime, date

##Data load

In [190]:
# Set URL to GitHub repo data (US confirmed cases and death)
url_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
url_death = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"

confirmed_data_raw = pd.read_csv(url_confirmed)
death_data_raw = pd.read_csv(url_death)

# DATA CLEANING

##Filter Confirmed Cases Data

In [191]:
# Drop unecessary columns in confirmed df
colms = ['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Lat', 'Long_', 'Country_Region']
confirmed_data_raw = confirmed_data_raw.drop(confirmed_data_raw[colms], axis=1)
state = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
              'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
              'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
              'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
              'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
              'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
              'West Virginia', 'Wisconsin', 'Wyoming', 'District of Columbia']
state_df = confirmed_data_raw[confirmed_data_raw['Province_State'].isin(state)]
state_df = state_df[pd.notna(state_df['FIPS'])]
state_df = state_df.rename(columns=({'Province_State':'US_State'}))
#state_df.head()

# Sum confirmed cases by state per day
state_sum_conf = state_df.groupby(['US_State']).sum()
state_sum_conf1 = state_sum_conf[state_sum_conf.columns[1:]]
# sum the total confimed cases under new column
state_sum_conf1['Confirmed_Totals'] = state_sum_conf1.sum(axis=1)
state_sum_conf1.reset_index(inplace=True)
state_sum_conf1

,US_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,Confirmed_Totals
0,Alabama,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,525049,525477,525898,526131,526348,526707,527083,527513,527922,528309,528597,528784,528784,529446,530011,530325,530744,530988,531094,531404,531751,532895,537813,539829,540083,540267,540603,540891,541230,541673,541928,542124,542256,542562,542831,543177,543405,543405,543405,106101077
1,Alaska,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,66909,67059,67059,67059,67509,67647,67834,68002,68148,68148,68148,68426,68518,68666,68762,68880,68880,68880,69117,69178,69267,69351,69464,69464,69464,69600,69657,69756,69835,69902,69902,69902,69999,70052,70119,70162,70208,70208,70208,11988507
2,Arizona,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,856451,857347,858076,858737,859487,860169,860772,861653,862497,863571,863927,864579,865280,866022,866623,867443,868382,868830,869472,870155,870624,871168,872022,872496,872978,873446,874065,874605,875195,875766,876411,876732,877182,877682,878338,879102,879909,880466,880466,167253521
3,Arkansas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,334222,334458,334642,334706,334769,334998,335289,335529,335725,335968,336105,336166,336462,336760,336948,337170,337415,337510,337586,337819,338043,338235,338485,338687,338782,338849,339162,339406,339662,339863,340040,340137,340186,340515,340698,340916,341130,341290,341351,65658407
4,California,0,0,0,0,2,2,2,2,2,3,3,3,6,6,6,6,6,6,6,6,7,7,8,8,8,8,8,8,8,8,10,10,10,10,10,10,11,11,12,...,3727913,3729116,3731145,3731770,3734562,3736745,3738327,3740038,3742115,3743681,3744937,3747337,3749580,3750644,3753173,3755869,3757115,3758137,3760303,3761779,3763281,3764405,3767052,3768087,3768805,3769827,3771240,3772702,3774494,3775619,3776346,3777077,3778555,3779998,3781120,3786459,3787911,3788713,3789227,700690012
5,Colorado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,496882,498758,500675,502242,503254,505268,506405,509194,510966,512804,513765,514721,516240,517851,519414,520816,522382,523309,524190,525474,526687,528533,530098,531070,531700,532389,533119,534364,535316,536128,536969,537401,537858,539261,540092,540924,541726,542405,542889,90672795
6,Connecticut,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,333732,334766,334766,334766,336933,337340,337961,338447,339233,339233,339233,340545,340962,341571,342282,342718,342718,342718,343545,343954,344141,344612,344977,344977,344977,345639,345720,345948,346154,346325,346325,346325,346711,346892,346980,347137,347341,347341,347341,60910061
7,Delaware,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,102267,102593,102927,103222,103459,103603,103844,104038,104405,104621,104909,105067,105204,105401,105572,105775,105948,106148,106252,106341,106455,106580,106715,106873,107007,107089,107175,107928,108031,108166,108280,108365,108409,108409,108532,108607,108681,108770,108821,18633426
8,District of Columbia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,47040,47141,47219,47323,47378,47471,47533,47614,47697,47697,47800,47903,47986,48041,48080,48136,48164,48180,48195,48282,48336,48404,48424,48530,48545,48590,48616,48634,48662,48689,48727,48748,48762,48803,48822,48844,48898,48898,48898,9926853
9,F

## Filter Death Data


In [180]:
# Drop unecessary columns and invalid states
cols = ['UID', 'iso2', 'iso3', 'code3','Lat', 'Long_', 'Admin2', 'Country_Region']
death_df = death_data_raw.drop(death_data_raw[cols], axis=1)
death_df = death_df.rename(columns=({'Province_State':'US_State'}))
death_df.reset_index(inplace=True)
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
              'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
              'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
              'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
              'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
              'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
              'West Virginia', 'Wisconsin', 'Wyoming', 'District of Columbia']
states_df = death_df[death_df['US_State'].isin(states)]
states_df = states_df[pd.notna(states_df['FIPS'])]
del states_df['FIPS']
#states_df.head()

# Sum all counties and aggregate by state and country (US)
state_sum = states_df.groupby(['US_State']).sum()
state_sum.reset_index(inplace=True)
del state_sum['index']
#state_sum1 = state_sum[state_sum.columns[:]]
#state_sum1['Death_Totals'] = state_sum1[2:].sum(axis=1)
state_sum

,US_State,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,...,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21
0,Alabama,4903185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10807,10824,10840,10851,10851,10852,10854,10879,10887,10896,10913,10913,10913,10913,10930,10946,10966,10978,10978,10978,10985,10997,11001,11024,11038,11038,11038,11043,11045,11077,11080,11112,11112,11112,11124,11138,11140,11146,11146,11146
1,Alaska,740995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,334,334,334,334,334,347,347,347,347,347,347,347,348,348,348,349,349,349,349,349,351,353,353,353,353,353,364,368,369,369,369,369,369,369,369,369,369,369,369,369
2,Arizona,7278717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17199,17221,17238,17260,17268,17268,17276,17282,17305,17324,17338,17344,17344,17355,17360,17367,17391,17407,17409,17409,17428,17430,17438,17447,17459,17459,17466,17480,17497,17509,17531,17547,17555,17555,17569,17576,17594,17609,17628,17628
3,Arkansas,3017804,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5708,5711,5716,5717,5718,5720,5725,5726,5735,5739,5741,5743,5747,5752,5754,5757,5759,5760,5761,5764,5770,5777,5783,5790,5793,5794,5796,5805,5805,5808,5811,5816,5817,5820,5824,5827,5829,5830,5830,5832
4,California,39512223,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,61193,61309,61401,61445,61479,61500,61526,61587,61672,61768,61866,61939,61967,62029,62070,62154,62220,62280,62314,62334,62365,62472,62573,62633,62656,62690,62699,62727,62765,62817,62862,62911,62933,62949,62986,63017,63168,63218,63236,63247
5,Colorado,5758736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6215,6225,6231,6249,6251,6255,6264,6272,6284,6295,6302,6304,6306,6321,6337,6345,6352,6355,6357,6357,6371,6402,6423,6432,6442,6450,6450,6456,6472,6478,6497,6503,6505,6509,6520,6531,6545,6552,6576,6580
6,Connecticut,3565287,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8027,8039,8047,8047,8047,8066,8067,8080,8084,8097,8097,8097,8112,8117,8124,8131,8137,8137,8137,8154,8156,8161,8168,8173,8173,8173,8194,8198,8204,8208,8212,8212,8212,8219,8221,8227,8230,8238,8238,8238
7,Delaware,973764,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1603,1604,1609,1615,1616,1617,1617,1622,1624,1625,1626,1626,1626,1626,1628,1629,1629,1636,1636,1641,1644,1644,1648,1651,1651,1651,1651,1651,1652,1652,1654,1659,1659,1659,1659,1660,1660,1660,1661,1661
8,District of Columbia,705749,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1097,1098,1099,1099,1099,1099,1100,1104,1105,1105,1105,1106,1106,1106,1109,1110,1110,1110,1110,1110,1113,1115,1115,1116,1118,1120,1122,1122,1124,1125,1125,1125,1127,1130,1130,1131,1132,1132,1132,1132
9,Florida,21477737,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,34616,34696,34759,34812,34848,34912,34958,35030,35084,35161,35239,35268,35307,35399,35478,35549,35635,35700,35731,35783,35831,35882,35929,36000,36056,36075,36133,36227,36271,36347,36441,36463,36474,36501,36581,36647,36733,36774,36774,36774


##Create a DF with June 2020 Confirmed Cases Data

In [122]:
# create df for month of June 2020
June_df = state_sum_conf1.loc[:,'6/1/20':'6/30/20']
June_df['June_Totals'] = June_df[June_df.columns[:]].sum(axis=1)

# DF with June 2020 total confirmed cases only
June_Totals_df = pd.DataFrame(June_df['June_Totals'])
June_Totals_df

,June_Totals
0,812738
1,23332
2,1294975
3,402154
4,4991174
5,886864
6,1348657
7,312601
8,291828
9,2637866


##Create a DF with population data only

In [184]:
# create a df with population data only
df_states = pd.DataFrame(state_sum['US_State'])
df_pop = pd.DataFrame(state_sum['Population'])
joined_df = df_states.join(df_pop) 
joined_df

Index(['US_State', 'Population'], dtype='object')

##Merge pop_df with June_Totals_df

In [187]:
# join both df
right = June_Totals_df
left = joined_df
merged_df = left.join(right)
merged_df

Index(['US_State', 'Population', 'June_Totals'], dtype='object')

# QUESTION 1:
##The most often used metric for overall rate of spread of the disease for a particular geography is cases per capita, which is the number of cases in that region divided by the population. Please calculate the infection rate per capita by state for June2021.

In [188]:
# per capita rate calculation of infection per 100k people
# x/total * 100000= infection PER 100K people
result = (merged_df['June_Totals']/merged_df['Population'] * 100000)
merged_df['Per_Capita (PER 100k)'] = result.round(0)
merged_df

,US_State,Population,June_Totals,Per_Capita (PER 100k)
0,Alabama,4903185,812738,16576.0
1,Alaska,740995,23332,3149.0
2,Arizona,7278717,1294975,17791.0
3,Arkansas,3017804,402154,13326.0
4,California,39512223,4991174,12632.0
5,Colorado,5758736,886864,15400.0
6,Connecticut,3565287,1348657,37827.0
7,Delaware,973764,312601,32102.0
8,District of Columbia,705749,291828,41350.0
9,Florida,21477737,2637866,12282.0


# QUESTION 2:
##Plot the infection rate by state (per capita as above) by day for 2021for the largest 4 states in the US; Florida, California, Texas, and New York. 

# QUESTION 3:
##Using the infections data and recent trends in how cases are moving, please come up with a methodology for forecasting the number of cases you expect to see in these 4 states for the next 90 days.

# QUESTION 4:
##Compare your forecasts to the one from the Institute of Health Metrics at the University of Washington (https://covid19.healthdata.org/projections)? How are your forecasts similar or different?